# Can Language Models produce correct structured output?
The goaal of this notebook is to explorer the nature of the mistakes a Language Modell can make when producing structured output like code. The idea is that it is prone to making syntactic errors that could be avoided when scaling up the dataset.

In [1]:
from src.RawDataLoaders import CodeSearchNet_RawDataLoader, CoNaLa_RawDataLoader
from src.models_and_transforms.text_transforms import Numericalise_Transform, Rename_Transform
from src.models_and_transforms.complex_transforms import *
from src.models_and_transforms.BART_models import BART_Simple
from src.Experiments import Sequence_BLEU_Experiment
from src.models_and_transforms.SOTA_transforms import CoNaLa_SOTA_Transform, Django_SOTA_Transform
from src.pipe_datasets import BART_Pipe_Dataset
from src.useful_utils import chunks, download_from_url, Validate_and_Save_Callback

from pytorch_lightning import Trainer, Callback, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateLogger
from pytorch_lightning.callbacks.base import Callback
from pytorch_lightning.loggers import WandbLogger
from tokenizers import processors, Tokenizer
from transformers import BartConfig
import torch
import ray
import wandb
# ray.init(ignore_reinit_error=True)
import random
import itertools
import cloudpickle
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import json
from tqdm.auto import tqdm 

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

## Running SOTA model on CoNaLa and Django

In [17]:
conala_RawDataLoader = CoNaLa_RawDataLoader()
conala_test_samples = conala_RawDataLoader.get_samples('test')
conala_transform = CoNaLa_SOTA_Transform(cuda=False, fields={'input_field':'description', 'output_field':'pred_code'})

load parser from [src/external_repos/external-knowledge-codegen/best_pretrained_models/finetune.mined.retapi.distsmpl.dr0.3.lr0.001.lr_de0.5.lr_da15.beam15.seed0.mined_100000.intent_count100k_topk1_temp5.bin]


loading reranker...


In [10]:
# takes around 5 mins for inference
samples = conala_transform(conala_test_samples[:10])

os.system('str_0')


initializing rerank features for hypotheses...
initializing features...
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.u

print(str_0.decode('str_0', 'ignore'))


initializing rerank features for hypotheses...
initializing features...
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.u

[i for i in var_0 if i == i]


...

initializing rerank features for hypotheses...
initializing features...
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.u

"""""".join([str_0])


initializing rerank features for hypotheses...
initializing features...
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.u

pd.concat([var_0] * 5, ignore_index=True)



initializing rerank features for hypotheses...
initializing features...
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.u

In [91]:
test_samples = [{'description':'multiply each element of list `foo` by 2'}]
test_samples = conala_transform(test_samples)
test_samples[0]['pred_code']

var_0 = [(i * j) for i, j in zip(var_0, var_0)]



initializing rerank features for hypotheses...
initializing features...
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.u

'foo = [(x * y) for x in foo]'

In [80]:
simi_expr = Sequence_BLEU_Experiment(fields={'target_seq':'code', 'predicted_seq':'pred_code'})
simi_expr.__transform__(samples)[:2]

[{'description': 'send a signal `signal.SIGUSR1` to the current process',
  'code': 'os.kill(os.getpid(), signal.SIGUSR1)',
  'pred_code': "os.system('<unk> <unk> <unk>')",
  'BLEU': 0.0},
 {'description': "decode a hex string '4a4b4c' to UTF-8.",
  'code': "bytes.fromhex('4a4b4c').decode('utf-8')",
  'pred_code': "print(4a4b4c.decode('4a4b4c'))",
  'BLEU': 0.0},
 {'description': 'check if all elements in list `myList` are identical',
  'code': 'all(x == myList[0] for x in myList)',
  'pred_code': '[i for i in myList if i == 2]',
  'BLEU': 0.0},
 {'description': 'format number of spaces between strings `Python`, `:` and `Very Good` to be `20`',
  'code': "print('%*s : %*s' % (20, 'Python', 20, 'Very Good'))",
  'pred_code': '""" """.join(str(x) for x in str(Python))',
  'BLEU': 0.0},
 {'description': 'How to convert a string from CP-1251 to UTF-8?',
  'code': "d.decode('cp1251').encode('utf8')",
  'pred_code': "unicodedata.normalize('NFKD', string).encode('utf8')",
  'BLEU': 0.27901593

In [93]:
samples = simi_expr(samples)

unsmoothed_official_BLEU: 0.22539098849201342
nltk_BLEU: 0.2529552430370321


In [94]:
json.dump(samples, open("CoNaLa_TranX_predictions.json", 'w'), indent=4)

# Training BART on CoNaLa
### Using the BPE tokens trained from code search net

# Training BART codeBPE

In [2]:
conala_RawDataLoader = CoNaLa_RawDataLoader()
conala_samples = conala_RawDataLoader.get_samples('mined_GPT3', max_char_len=200)[:200000]
print('Dataset size:',len(conala_samples))
conala_samples[31]

Dataset size: 200000


{'description': 'sort a list of lists `the_list` by the second item in each list',
 'code': "sorted(the_list, key=lambda x: int(x.split('_')[1]))"}

In [ ]:
code_numericalise_transform = Numericalise_Transform(numericaliser='Code32k', fields=[('description','input_ids'),('code','target_ids')])

conala_dataset = BART_Pipe_Dataset(conala_samples, 
                                         sort_key_fn=lambda sample_obj: len(sample_obj['target_ids']),
#                                          valid_sample_fn=lambda sample_obj: len(sample_obj['target_ids'])<80 and len(sample_obj['input_ids'])<80,
                                         shuffle=True,
                                         batch_bucket_size=1, 
                                         slow_pipe=[], real_time_pipe=[code_numericalise_transform])
conala_dataloader = conala_dataset.to_dataloader(16)
lengths = [len(s['target_ids']) for s in conala_dataset]
plt.scatter(range(len(lengths)), lengths)

Numericaliser. Ex: 'This is a test' -> [0, 3863, 343, 276, 1772, 2]


In [ ]:
config = BartConfig(vocab_size=32000)
model = BART_Simple(from_pretrained=False, config=config)

In [2]:
def CoNaLa_valid_fn(model):
    conala_RawDataLoader = CoNaLa_RawDataLoader()
    BART_cond_gen_transform = BART_Conditional_Generator_Transform(model, 
                                                                    numericaliser='Code32k', 
                                                                    denumericaliser='Code32k',
                                                                    device='cuda',
                                                                    debug=False,
                                                                    chunk_size=32,
                                                                    max_length=40)
    valid_samples = conala_RawDataLoader.get_samples('valid')
    test_samples = conala_RawDataLoader.get_samples('test')
    train_samples = conala_RawDataLoader.get_samples('mined_GPT3')[:100]
    return_dict = {}
    
    for samples, name in [(valid_samples, 'valid'), (test_samples, 'test'), (train_samples, 'train')]:
        samples = Rename_Transform(fields=[('description', 'input_text')])(samples)
        samples = BART_cond_gen_transform(samples)
        samples = Sequence_BLEU_Experiment(fields = {'predicted_seq':'pred_text', 'target_seq':'code'}, debug=False)(samples)
        
        nltk_BLEU = np.average([s["nltk_BLEU"] for s in samples])
        table = wandb.Table(columns=["Desc", "True", "Pred"])
        for sample_obj in samples:
            table.add_data(sample_obj['input_text'], sample_obj['code'], sample_obj['pred_text'])
        
        return_dict[f'{name}_nltk_BLEU'] = nltk_BLEU
        return_dict[f'{name}_samples'] = table
    
    return return_dict

In [ ]:
saving_cb = Validate_and_Save_Callback(filepath='saved_models/BART_CoNaLa/',
                                       prefix='BART_codeBPE_mined200k_step_LR',
                                       validate_fn=CoNaLa_valid_fn,
                                       monitor='valid_nltk_BLEU',
                                       interval=0.1)
wandb_logger = WandbLogger(name='BART_codeBPE_mined200k_step_LR',project='pytorchlightning')
lr_logger_cb = LearningRateLogger(logging_interval='step')

trainer = Trainer(gpus=[0], gradient_clip_val=0.5, amp_level='O1', max_epochs=200, callbacks=[saving_cb, lr_logger_cb], logger=wandb_logger, 
                  accumulate_grad_batches=1)
trainer.fit(model, conala_dataloader)

In [70]:
torch.save({'model':model, 'train_dataset':train_dataset}, 'BART_6k_chars.pickle', pickle_module=cloudpickle)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BART_Simple. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BartForConditionalGeneration. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BartModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "

In [8]:
config = BartConfig(vocab_size=32000)
BART_cond_gen_transform = BART_Conditional_Generator_Transform("saved_models/BART_CoNaLa/BART_codeBPE_mined20k_set_lowLR_{'nltk_BLEU': 0.2149831961314364, 'valid_samples': <wandb.data_types.Table object at 0x7f669336c1d0>}_step_64875.ckpt", 
                                                                numericaliser='Code32k', 
                                                                denumericaliser='Code32k',
                                                                device='cuda',
                                                                from_pretrained=False,
                                                                config=config)

Numericaliser. Ex: 'This is a test' -> [0, 3863, 343, 276, 1772, 2]
Denumericaliser. Ex: [0,1,2,3,4,5,6,7,8,9] -> !"#$%


In [23]:
conala_RawDataLoader = CoNaLa_RawDataLoader()
# test_samples = conala_RawDataLoader.get_samples('train')[:10]
# test_samples = conala_samples[:10]
test_samples = [{'description':'split a string `foo` by comma'}]
test_samples = Rename_Transform(fields=[('description', 'input_text')])(test_samples)

test_samples = BART_cond_gen_transform(test_samples)
test_samples

[{'input_text': 'split a string `foo` by comma',
  'input_ids': [0, 961, 276, 711, 613, 3797, 68, 626, 6155, 2],
  'pred_ids': [0, 3797, 277, 6773, 18, 961, 401, 3797, 4022, 438],
  'pred_text': "foo = foo.split('foo bar')"}]

# Pre-Training BART on Code Search Net

In [2]:
code_search_net_RawDataLoader = CodeSearchNet_RawDataLoader("./datasets/code_search_net", language="python", max_chars=600)
pretrain_samples = code_search_net_RawDataLoader.get_samples("train", fields=['code'])

In [3]:
pretrain_samples[0]

{'code': 'def _rect_to_css(rect):\n    """\n    Convert a dlib \'rect\' object to a plain tuple in (top, right, bottom, left) order\n\n    :param rect: a dlib \'rect\' object\n    :return: a plain tuple representation of the rect in (top, right, bottom, left) order\n    """\n    return rect.top(), rect.right(), rect.bottom(), rect.left()'}

In [4]:
pretrain_dataset = BART_Pipe_Dataset(pretrain_samples, 
                                     sort_key_fn=lambda sample_obj: len(sample_obj['target_ids']),
                                     valid_sample_fn=lambda sample_obj: len(sample_obj['target_ids'])<200 and len(sample_obj['input_ids'])<200,
                                     shuffle=True,
                                     batch_bucket_size=1, slow_pipe=[], real_time_pipe=[
        BART_Corrupt_Augmentation_Live_Transform(corruption_types={"span_deletion":{'min_tokens':1,'max_tokens':10, 'deletion_ratio':0.15}},
                                                 fields={"input_seq":"code", "augmented_seq":"corrupted_code"},
                                                 display_bar=False),
        Numericalise_Transform(numericaliser='Code32k', fields=[('corrupted_code','input_ids'),('code','target_ids')])
    ])
print(f'filtered dataset length: {len(pretrain_dataset)}')
lengths = [len(s['target_ids']) for s in tqdm(pretrain_dataset, desc='Plotting lengths')]
plt.scatter(range(len(lengths)), lengths)
plt.title("Length distribution of target samples in dataset")
plt.show()

Numericaliser. Ex: 'This is a test' -> [0, 3863, 343, 276, 1772, 2]



filtered dataset length: 183677


KeyboardInterrupt: 

In [5]:
print('### Corrupted code: ###')
print(pretrain_dataset[0]['corrupted_code'])
print('---------------')
print('### Fixed code: ###')
print(pretrain_dataset[0]['code'])

### Corrupted code: ###
def validate_supersmoother_bass():
    """Validate toother with extra bass.""  x,= smoother_friedd_samoother_problem_friedman82()figure()
    plt.plot(x, y, '.', label='Data')
    for bass in range(0, 10, 3):
        smoothupersmoother.SuperSmoother()
    mooth.set_bass_enhancement(bass)
        smooth.specify_data_set(x, y)
        smooth.compute()
        plt.plot(x, smooth.smh_result, '.', label='Bass = at(bass))
        # pylab.plot(selother.smoott, labl='Bass = {0}'.foass)) finish_plot()
---------------
### Fixed code: ###
def validate_supersmoother_bass():
    """Validate the supersmoother with extra bass."""
    x, y = smoother_friedman82.build_sample_smoother_problem_friedman82()
    plt.figure()
    plt.plot(x, y, '.', label='Data')
    for bass in range(0, 10, 3):
        smooth = supersmoother.SuperSmoother()
        smooth.set_bass_enhancement(bass)
        smooth.specify_data_set(x, y)
        smooth.compute()
        plt.plot(x, smooth.smooth_resul

In [6]:
pretrain_dataloader = pretrain_dataset.to_dataloader(16)

In [7]:
config = BartConfig(vocab_size=32000)
model = BART_Simple(from_pretrained=False, config=config)

In [8]:
def CodeSearchNet_valid_fn(model):
    BART_cond_gen_transform = BART_Conditional_Generator_Transform(model, 
                                                                    numericaliser='Code32k', 
                                                                    denumericaliser='Code32k',
                                                                    device='cuda',
                                                                    debug=False,
                                                                    chunk_size=8,
                                                                    max_length=200)
    
    code_search_net_RawDataLoader = CodeSearchNet_RawDataLoader("./datasets/code_search_net", language="python", max_chars=600)
    valid_samples = code_search_net_RawDataLoader.get_samples("valid", fields=['code'])[:100]
    train_samples = code_search_net_RawDataLoader.get_samples("train", fields=['code'])[:100]
    
    valid_samples = BART_Pipe_Dataset(valid_samples, 
                                     sort_key_fn=lambda sample_obj: len(sample_obj['input_ids']),
                                     valid_sample_fn=lambda sample_obj: len(sample_obj['input_ids'])<200 and len(sample_obj['input_ids'])<200,
                                     shuffle=True,
                                     batch_bucket_size=1, slow_pipe=[], real_time_pipe=[
        BART_Corrupt_Augmentation_Live_Transform(corruption_types={"span_deletion":{'min_tokens':1,'max_tokens':10, 'deletion_ratio':0.15}},
                                                 fields={"input_seq":"code", "augmented_seq":"corrupted_code"},
                                                 display_bar=False),
        Numericalise_Transform(numericaliser='Code32k', fields=[('corrupted_code','input_ids')])
    ])
    
    train_samples = BART_Pipe_Dataset(train_samples, 
                                     sort_key_fn=lambda sample_obj: len(sample_obj['input_ids']),
                                     valid_sample_fn=lambda sample_obj: len(sample_obj['input_ids'])<200 and len(sample_obj['input_ids'])<200,
                                     shuffle=True,
                                     batch_bucket_size=1, slow_pipe=[], real_time_pipe=[
        BART_Corrupt_Augmentation_Live_Transform(corruption_types={"span_deletion":{'min_tokens':1,'max_tokens':10, 'deletion_ratio':0.15}},
                                                 fields={"input_seq":"code", "augmented_seq":"corrupted_code"},
                                                 display_bar=False),
        Numericalise_Transform(numericaliser='Code32k', fields=[('corrupted_code','input_ids')])
    ])
    
    return_dict = {}
    
    for samples, name in [(valid_samples, 'valid'), (train_samples, 'train')]:
        samples = Rename_Transform(fields=[('corrupted_code', 'input_text')])(samples)
        samples = BART_cond_gen_transform(samples)
        samples = Sequence_BLEU_Experiment(fields = {'predicted_seq':'pred_text', 'target_seq':'code'}, debug=False)(samples)
        
        nltk_BLEU = np.average([s["nltk_BLEU"] for s in samples])
        table = wandb.Table(columns=["Corrupted", "True", "Pred"])
        for sample_obj in samples:
            table.add_data(sample_obj['input_text'], sample_obj['code'], sample_obj['pred_text'])
        
        return_dict[f'{name}_nltk_BLEU'] = nltk_BLEU
        return_dict[f'{name}_samples'] = table
    
    return return_dict

In [16]:
saving_cb = Validate_and_Save_Callback(filepath='saved_models/BART_CodeSearchNet/',
                                       prefix='BART_codeBPE_CSN_150k_bigLR',
                                       validate_fn=CodeSearchNet_valid_fn,
                                       monitor='valid_nltk_BLEU',
                                       interval=0.3,
                                       skip_epochs=[])
wandb_logger = WandbLogger(name='BART_codeBPE_CSN_150k_v1.0_bigLR',project='code_pretraining')
lr_logger_cb = LearningRateLogger(logging_interval='step')

In [17]:
model.lr = 0.00005

In [18]:
torch.save({'model':model, 
            'train_dataset':pretrain_dataset, 
            'callbacks':[lr_logger_cb, saving_cb],
            'logger':wandb_logger
           }, "BART_CSN_codeBPE_150k_bigLR_v1.0.pickle", pickle_module=cloudpickle)

In [3]:
load_obj = torch.load("BART_CSN_codeBPE_1k_v1.1.pickle")
model = load_obj['model']
train_dataset = load_obj['train_dataset']
callbacks = load_obj['callbacks']
logger = load_obj['logger']

train_dataloader = train_dataset.to_dataloader(16)

trainer = Trainer(gpus=[0], num_nodes=1, gradient_clip_val=0.5, amp_level='O1', callbacks=callbacks, logger=logger)
trainer.fit(model, train_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
wandb: Currently logged in as: aquaktus (use `wandb login --relogin` to force relogin)


Problem at: /usr/local/lib/python3.6/dist-packages/pytorch_lightning/loggers/wandb.py 130 experiment


KeyboardInterrupt: 

In [ ]:
saving_cb = Validate_and_Save_Callback(filepath='saved_models/BART_CodeSearchNet/',
                                       prefix='BART_codeBPE_CSN_1k',
                                       validate_fn=CodeSearchNet_valid_fn,
                                       monitor='valid_nltk_BLEU',
                                       interval=0.1,
                                       skip_epochs=[0,1,2])
wandb_logger = WandbLogger(name='BART_codeBPE_CSN_1k_v1.1',project='code_pretraining')
lr_logger_cb = LearningRateLogger(logging_interval='step')

# Remember ddp is not available from jupyter, only bash shell
trainer = Trainer(gpus=[0], gradient_clip_val=0.5, amp_level='O1', max_epochs=200, callbacks=[saving_cb, lr_logger_cb], logger=wandb_logger)
trainer.fit(model, pretrain_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0,1]


In [16]:
trainer.save_checkpoint('saved_models/BART_CoNaLa/BART_CodeBPE_CoNaLa_100_epochs')

In [24]:
BART_cond_gen_transform = BART_Conditional_Generator_Transform('saved_models/BART_CodeSearchNet/BART_codeBPE_CSN_150k_bigLR_0.86610_step_234192.ckpt', 
                                                                    numericaliser='Code32k', 
                                                                    denumericaliser='Code32k',
                                                                    device='cuda',
                                                                    debug=False,
                                                                    chunk_size=8,
                                                                    config=BartConfig(vocab_size=32000),
                                                                    max_length=200)

In [51]:
test_samples = [
    {'input_text':'''
def is_integer_array(val): 
    """ Checks whether a variable is a numpy integer array. 
    Inputs 
    --- val The variable to check. 
    Returns ------- bool True if the variable is a numpy integer array. 
    Otherwise False. """
    i'''}
]
test_samples = BART_cond_gen_transform(test_samples)
print(test_samples[0]['pred_text'])


def is_integer_array(val): 
    """ Checks whether a variable is a numpy integer array. 
    
    Inputs
    ----------
    val: The variable to check. 
    
    Returns
    -------
    bool
        True if the variable is a numpy integer array. 
    Otherwise False.
    """
    return val


In [54]:
test_samples = [{'input_text':"foo is an empty list"}]
print(BART_cond_gen_transform(test_samples)[0]['pred_text'])


def is_list(foo, *empty_list):
  'A empty list'
  return True if foo is an empty list and list(foo) is an empty list or list(foo) is an empty list'
  return foo is an empty list or list(foo) is an empty list or list(empty_list)


# Fine Tunning CodeBART on CoNaLa
First step is to get the best checkpoint from training on Code Search net and loading it. I want to make sure when i fine tune that I have some warm up steps to not forget everything.

In [3]:
config = BartConfig(vocab_size=32000)
model = BART_Simple(from_pretrained=False, config=config)

In [ ]:
ckpt = torch.load('saved_models/BART_CodeSearchNet/BART_codeBPE_CSN_150k_bigLR_0.86215_step_165312.ckpt')
save_dict = ckpt['state_dict']

In [ ]:
model.load_state_dict(save_dict)

Let's override the optimiser step to include some warmup

In [39]:
# enabel this in the model

Getting the CoNaLa data

In [27]:
conala_RawDataLoader = CoNaLa_RawDataLoader()
conala_samples = conala_RawDataLoader.get_samples('train', max_char_len=200)[:200000]
print('Dataset size:',len(conala_samples))
conala_samples[31]

Dataset size: 2139


{'description': 'Extract values not equal to 0 from numpy array `a`',
 'code': 'a[a != 0]'}

In [28]:
code_numericalise_transform = Numericalise_Transform(numericaliser='Code32k', fields=[('description','input_ids'),('code','target_ids')])
conala_dataset = BART_Pipe_Dataset(conala_samples, 
                                         sort_key_fn=lambda sample_obj: len(sample_obj['target_ids']),
#                                          valid_sample_fn=lambda sample_obj: len(sample_obj['target_ids'])<80 and len(sample_obj['input_ids'])<80,
                                         shuffle=True,
                                         batch_bucket_size=1, 
                                         slow_pipe=[], real_time_pipe=[code_numericalise_transform])
conala_dataloader = conala_dataset.to_dataloader(16)

Numericaliser. Ex: 'This is a test' -> [0, 3863, 343, 276, 1772, 2]


### make sure to run the validation function cell from above for the CoNaLa validation `CodeSearchNet_valid_fn`

In [32]:
saving_cb = Validate_and_Save_Callback(filepath='saved_models/BART_CoNaLa/',
                                       prefix='BART_CSN_pretrained_CoNaLa_train_v1.0',
                                       validate_fn=CoNaLa_valid_fn,
                                       monitor='valid_nltk_BLEU',
                                       interval=0.3,
                                       skip_epochs=[])
wandb_logger = WandbLogger(name='BART_CSN_pretrained_CoNaLa_train_v1.0',project='CoNaLa_models')
lr_logger_cb = LearningRateLogger(logging_interval='step')

In [42]:
trainer = Trainer(gpus=[0], num_nodes=1, gradient_clip_val=0.5, amp_level='O1', callbacks=[saving_cb, lr_logger_cb], logger=wandb_logger)
trainer.fit(model, conala_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name | Type                         | Params
------------------------------------------------------
0 | BART | BartForConditionalGeneration | 387 M 


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Saving latest checkpoint..


1

### Saving to pickle

In [30]:
torch.save({'model':model, 
            'train_dataset':conala_dataset, 
            'callbacks':[lr_logger_cb, saving_cb],
            'logger':wandb_logger
           }, "pickle_train_files/BART_CSN_pretrained_CoNaLa_train_v1.0.pickle", pickle_module=cloudpickle)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BART_Simple. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LightningModule. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BartForConditionalGeneration. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BartModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be che